In [2]:
import subprocess

In [ ]:
def getSampleMatrix():
    script = "./drosChr2L.sh"
    n_samples = 3
    n_samples_per_run = 1
    is_download = True
    try:
        result = subprocess.run(
            ["bash", script, str(n_samples), str(n_samples_per_run), str(is_download)],
            capture_output=True,
            text=True,
            check=True,
        )

        print("Output:", result.stdout)
        if result.stderr:
            print("Error:", result.stderr)

    except subprocess.CalledProcessError as e:
        print("Script failed with error:", e.stderr)


getSampleMatrix()

Output: Processing file 1 of 2: GM.chr1.13000000.13075000.txt
Running command: ../bin/sBIF -i ../data/folding_input/GM.chr1.13000000.13075000.txt -c chr1 -l ../data/chromosome_sizes.txt -s 13000000 -e 13075000 -ns 3 -nr 1 -cl GM -o ../output -r 5000 -do False -j chr1 -p 50
Trimmed item value: 'False'
Parameters: 
Interaction file :../data/folding_input/GM.chr1.13000000.13075000.txt
Chromosome :chr1
Chrom lengths file :../data/chromosome_sizes.txt
Start position:13000000
End position :13075000
Cell line :GM
Download :0
Output folder :../output
Resolution :5000
Fiber density :0.2368
Number of samples :3
Number of samples per run :1
Number of potential sphere points :50
Knock-in distance :80
Maximum trials :100
Number of iteractions :100
Job prefix :chr1
Number of threads :50
Bead diameter: 34.2923
Generating samples ...
Inserted GM: chr1.13000000-13075000 (15 samples) successfully.
Inserted GM: chr1.13000000-13075000 (15 samples) successfully.
Inserted GM: chr1.13000000-13075000 (15 samp